# Classifying YouTube Videos for Humpback Whale Encounters - Keras CNN-RNN

In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import pickle
import glob
import cv2
import os

from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
import seaborn as sns

import wandb

from data_preprocessing import get_video_frames, frame_capture

In [ ]:
#start wandb session for metric logging
wandb.login() 

wandb.init(project="whale-classification-inception")

# Inception V3 (CNN-RNN) 

## Hyperparameters

In [3]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_NUM_FRAMES = 500
NUM_FEATURES = 2048

In [19]:
frames = get_video_frames('video_clip_0000.mp4', resize=(IMG_SIZE, IMG_SIZE))

num frames extracted: 361


In [36]:
num_frames = frame_capture('/mount/data/video_clips/video_clip_0000.mp4')
print(num_frames)

473.0
frame 361 not available
frame 362 not available
frame 363 not available
frame 364 not available
frame 365 not available
frame 366 not available
frame 367 not available
frame 368 not available
frame 369 not available
frame 370 not available
frame 371 not available
frame 372 not available
frame 373 not available
frame 374 not available
frame 375 not available
frame 376 not available
frame 377 not available
frame 378 not available
frame 379 not available
frame 380 not available
frame 381 not available
frame 382 not available
frame 383 not available
frame 384 not available
frame 385 not available
frame 386 not available
frame 387 not available
frame 388 not available
frame 389 not available
frame 390 not available
frame 391 not available
frame 392 not available
frame 393 not available
frame 394 not available
frame 395 not available
frame 396 not available
frame 397 not available
frame 398 not available
frame 399 not available
frame 400 not available
frame 401 not available
frame 402 

In [37]:
#remove frames
import os
import glob

files = glob.glob('/workspace/youtube-humpback-whale-classifier/classification/frames/frame*.jpg')
for f in files:
    os.remove(f)

In [82]:
import os
import subprocess

def convert_mp4_to_avi(mp4_file):
    avi_file_path = '/mount/data/' + mp4_file.replace('.mp4', '.avi')
    mp4_file_path = '/mount/data/video_clips/' + mp4_file
    
    print(avi_file_path)
    print(mp4_file_path)
    subprocess.call(f"ffmpeg -loglevel panic -i {mp4_file_path} -c:a copy -c:v h264 {avi_file_path} ", shell=True) #convert mp4 to avi while keeping h264 codec

    return True

mp4_video = 'video_clip_0003.mp4'
avi_video = '/mount/data/video_clip_0003.avi'
convert_mp4_to_avi(mp4_video)

/mount/data/video_clip_0003.avi
/mount/data/video_clips/video_clip_0003.mp4


True

In [74]:
def convert_mp4_to_h264(mp4_file):
    mp4_file_path = '/mount/data/video_clips/' + mp4_file
    h264_path = '/mount/data/' + mp4_file.replace('.mp4', '.h264')
    
    command = "MP4Box -add {mp4_path} {h264_path}"
    subprocess.call([command], shell=True)
    print("vid conv")
    
mp4_video = 'video_clip_0003.mp4'
convert_mp4_to_h264(mp4_video)

vid conv


/bin/dash: 1: MP4Box: not found


In [69]:
#get frame count - avi
cap = cv2.VideoCapture('/mount/data/video_clip_0003.avi')
cap.get(7)

451.0

In [70]:
#get frame count - mp4
cap = cv2.VideoCapture('/mount/data/video_clips/' + mp4_video)
cap.get(7)

456.0

In [71]:
num_frames = frame_capture('/mount/data/video_clips/video_clip_0003.mp4')
print(num_frames)

456.0
frame 450 not available
frame 451 not available
frame 452 not available
frame 453 not available
frame 454 not available
frame 455 not available
456


In [83]:
import subprocess

cmd = "ffprobe -i {video_file}".format(video_file = '/mount/data/video_clip_0003.avi')
# cmd = "ffprobe -i {video_file}".format(video_file = '/mount/data/video_clips/video_clip_0003.mp4')

subprocess.call(cmd, shell=True)

ffprobe version 4.2.7-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --

0

In [84]:
import subprocess

cmd = "ffprobe -i {video_file}".format(video_file = '/mount/data/video_clip_0002.avi')
# cmd = "ffprobe -i {video_file}".format(video_file = '/mount/data/video_clips/video_clip_0003.mp4')

subprocess.call(cmd, shell=True)

ffprobe version 4.2.7-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --

0